In [ ]:
from draw import plot_2d_array
from Arguments import Arguments

task = 'FASHION'
if task != 'MOSI':
    from QWAS_MNIST import *
    arch_code = [4, 4]  
else:
    from QWAS_MOSI import *
    arch_code = [7, 5]

In [ ]:
# set random seed
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
mp.set_start_method('spawn')
arg = Arguments()

In [ ]:
# SuperBase
single = [[i]+[1]*8 for i in range(1,5)]
enta = [[i]+[i+1]*4 for i in range(1,4)]+[[4]+[1]*4]

In [ ]:
# Random

# empty = empty_arch(4, 4)
# single = random.sample(search_space_single, 2)
# enta = random.sample(search_space_enta, 2)
# single = agent.insert_job(empty[0], single)
# enta = agent.insert_job(empty[1], enta)

In [ ]:
check_file(task)
arch = cir_to_matrix(single, enta, arch_code)
plot_2d_array(arch)

In [ ]:
agent = create_agent(task, arch_code)
# agent.explorations['rate'] = arg.exploration
ITERATION = agent.ITERATION
num_processes = 5 

In [ ]:
for iter in range(ITERATION, 55):
        jobs, designs, archs, nodes = agent.early_search(iter)
        results = {}
        n_jobs = len(jobs)
        step = n_jobs // num_processes
        res = n_jobs % num_processes
        if task != 'MOSI':
            with Manager() as manager:
                q = manager.Queue()
                with mp.Pool(processes = num_processes) as pool:        
                    pool.starmap(Scheme_mp, [(designs[i*step : (i+1)*step], task, agent.weight, i, q) for i in range(num_processes)])            
                    pool.starmap(Scheme_mp, [(designs[n_jobs-i-1 : n_jobs-i], task, agent.weight, n_jobs-i-1, q) for i in range(res)])
                while not q.empty():
                    [i, acc] = q.get()
                    results[i] = acc
        else:
            for i in range(n_jobs):
                _, report = Scheme(designs[i], task, agent.weight, 3, verbs=1)
                results[i] = report['mae']

        agent.late_search(jobs, results, archs, nodes)

print('The best model: ', agent.best['acc'])
plot_2d_array(agent.best['model'])